In [1]:
import pandas as pd

df1 = pd.read_csv('SKW_final_w_Pd.csv')

print("DataFrame Head:")
display(df1.head())

print("\nDataFrame Info:")
display(df1.info())

DataFrame Head:


,Location,UWI,Reading Time,Average Amps (A) (Raw),Drive Frequency (Hz) (Raw),Intake Pressure (psi) (Raw),Discharge Pressure (psi) (Raw),Intake Temperature (F) (Raw),Motor Temperature (F) (Raw),Vibration (gravit) (Raw),Virtual Rate (BFPD) (Raw)
0,SKW-33,SKW-33,2025-03-03 17:27:15,0.0,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
1,SKW-33,SKW-33,2025-03-03 17:40:00,0.0,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
2,SKW-33,SKW-33,2025-03-03 17:50:00,0.0,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
3,SKW-33,SKW-33,2025-03-03 18:00:00,0.0,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
4,SKW-33,SKW-33,2025-03-03 18:10:00,0.0,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19449 entries, 0 to 19448
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Location                        19449 non-null  object 
 1   UWI                             19449 non-null  object 
 2   Reading Time                    19449 non-null  object 
 3   Average Amps (A) (Raw)          19449 non-null  float64
 4   Drive Frequency (Hz) (Raw)      19449 non-null  float64
 5   Intake Pressure (psi) (Raw)     19449 non-null  float64
 6   Discharge Pressure (psi) (Raw)  19449 non-null  float64
 7   Intake Temperature (F) (Raw)    19449 non-null  float64
 8   Motor Temperature (F) (Raw)     19449 non-null  float64
 9   Vibration (gravit) (Raw)        19449 non-null  float64
 10  Virtual Rate (BFPD) (Raw)       19449 non-null  float64
dtypes: float64(8), object(3)
memory usage: 1.6+ MB


None

In [2]:
df = df1.drop(df1.columns[4], axis=1)

In [3]:
df['Reading Time'] = pd.to_datetime(df['Reading Time'])
df = df.sort_values(by='Reading Time')
display(df.head())

,Location,UWI,Reading Time,Average Amps (A) (Raw),Intake Pressure (psi) (Raw),Discharge Pressure (psi) (Raw),Intake Temperature (F) (Raw),Motor Temperature (F) (Raw),Vibration (gravit) (Raw),Virtual Rate (BFPD) (Raw)
0,SKW-33,SKW-33,2025-03-03 17:27:15,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
1,SKW-33,SKW-33,2025-03-03 17:40:00,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
2,SKW-33,SKW-33,2025-03-03 17:50:00,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
3,SKW-33,SKW-33,2025-03-03 18:00:00,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
4,SKW-33,SKW-33,2025-03-03 18:10:00,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0


In [4]:
import numpy as np
from scipy.stats import linregress

# --- sort dataframe ---
df = df.sort_values('Reading Time').reset_index(drop=True)

# --- tentukan interval waktu (30 menit) ---
time_interval = pd.Timedelta(minutes=30)

# snap waktu awal & akhir ke kelipatan 30 menit
start_time = df['Reading Time'].iloc[0].floor('30min')
end_time = df['Reading Time'].iloc[-1].ceil('30min')

# buat daftar window tiap 30 menit
time_windows = pd.date_range(start=start_time, end=end_time, freq='30min')

# --- hitung slope ---
slopes_list = []

for window_start in time_windows:
    window_end = window_start + time_interval
    window_df = df[(df['Reading Time'] >= window_start) & (df['Reading Time'] < window_end)]

    # skip kalau data terlalu sedikit
    if len(window_df) < 2:
        continue

    # ambil kolom numerik
    numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
    cols_to_exclude = ['Discharge Pressure (psi) (Raw)']  # contoh kalau mau dikecualikan
    # numerical_cols = [col for col in numerical_cols if col not in cols_to_exclude]

    # hitung slope per kolom
    window_slopes = {}
    for col in numerical_cols:
        temp_df = window_df[['Reading Time', col]].dropna()
        if len(temp_df) > 1:
            temp_time_diff = (temp_df['Reading Time'] - window_start).dt.total_seconds().values
            slope, _, _, _, _ = linregress(temp_time_diff, temp_df[col])
            window_slopes[col + '_slope'] = slope
        else:
            window_slopes[col + '_slope'] = np.nan

    # simpan waktu window
    window_slopes['Window_Start_Time'] = window_start
    slopes_list.append(window_slopes)

# --- konversi ke DataFrame ---
slopes_df = pd.DataFrame(slopes_list)

# jadikan Window_Start_Time sebagai index
slopes_df = slopes_df.set_index('Window_Start_Time')


In [5]:
# from scipy.stats import linregress
# import numpy as np


# # Tentukan interval waktu (30 menit)
# time_interval = pd.Timedelta(hours=0.5)

# # Urutkan DataFrame berdasarkan waktu
# df = df.sort_values('Reading Time').reset_index(drop=True)

# # Buat list semua titik waktu awal yang kelipatan 30 menit dari waktu awal
# start_time = df['Reading Time'].iloc[0]
# end_time = df['Reading Time'].iloc[-1]
# time_windows = pd.date_range(start=start_time, end=end_time, freq='0.5H')

# # Inisialisasi list untuk menyimpan hasil slope
# slopes_list = []

# # Iterasi setiap window waktu
# for window_start in time_windows:
#     window_end = window_start + time_interval
#     window_df = df[(df['Reading Time'] >= window_start) & (df['Reading Time'] < window_end)]

#     # Lewati window jika datanya terlalu sedikit
#     if len(window_df) < 2:
#         continue

#     # Ekstrak kolom numerik
#     numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
#     cols_to_exclude = ['Discharge Pressure (psi) (Raw)']  # jika ingin mengecualikan
#     # numerical_cols = [col for col in numerical_cols if col not in cols_to_exclude]

#     # Hitung slope tiap kolom numerik
#     window_slopes = {}
#     for col in numerical_cols:
#         temp_df = window_df[[col, 'Reading Time']].dropna()
#         if len(temp_df) > 1:
#             temp_time_diff = (temp_df['Reading Time'] - window_start).dt.total_seconds().values
#             slope, _, _, _, _ = linregress(temp_time_diff, temp_df[col])
#             window_slopes[col + '_slope'] = slope
#         else:
#             window_slopes[col + '_slope'] = np.nan

#     window_slopes['Window_Start_Time'] = window_start
#     slopes_list.append(window_slopes)

# # Konversi ke DataFrame
# slopes_df = pd.DataFrame(slopes_list)

# display(slopes_df.head())

In [6]:
# Buat salinan daftar nama kolom
cols1 = slopes_df.columns.tolist()

cols1[0] = 'A'
cols1[1] = 'IP'
cols1[2] = 'DP'   
cols1[3] = 'IT'   
cols1[4] = 'MT'   
cols1[5] = 'V'   
cols1[6] = 'R'

# Terapkan kembali ke dataframe
slopes_df.columns = cols1

In [7]:
df

,Location,UWI,Reading Time,Average Amps (A) (Raw),Intake Pressure (psi) (Raw),Discharge Pressure (psi) (Raw),Intake Temperature (F) (Raw),Motor Temperature (F) (Raw),Vibration (gravit) (Raw),Virtual Rate (BFPD) (Raw)
0,SKW-33,SKW-33,2025-03-03 17:27:15,0.0,0.0,0.000000,-4.000000e-09,-4.000000e-09,0.0,0.0
1,SKW-33,SKW-33,2025-03-03 17:40:00,0.0,0.0,0.000000,-4.000000e-09,-4.000000e-09,0.0,0.0
2,SKW-33,SKW-33,2025-03-03 17:50:00,0.0,0.0,0.000000,-4.000000e-09,-4.000000e-09,0.0,0.0
3,SKW-33,SKW-33,2025-03-03 18:00:00,0.0,0.0,0.000000,-4.000000e-09,-4.000000e-09,0.0,0.0
4,SKW-33,SKW-33,2025-03-03 18:10:00,0.0,0.0,0.000000,-4.000000e-09,-4.000000e-09,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
19444,SKW-33,SKW-33,2025-05-28 09:00:00,0.0,2527.8,1546.633378,2.631000e+02,2.642000e+02,0.2,0.0
19445,SKW-33,SKW-33,2025-05-28 10:00:00,0.0,2527.6,1546.633378,2.631000e+02,2.644000e+02,0.2,0.0
19446,SKW-33,SKW-33,2025-05-28 11:00:00,0.0,2527.5,1546.633378,2.631000e+02,2.642000e+02,0.2,0.0
19447,SKW-33,SKW-33,2025-05-28 12:00:00,0.0,2528.0,1546.633378,2.631000e+02,2.642000e+02,0.2,0.0


In [8]:
# # Pastikan kolom Reading Time sudah datetime
# df1["Reading Time"] = pd.to_datetime(df1["Reading Time"])

# # Tentukan start time dari data pertama
# start_time = df1["Reading Time"].min()

# # Set index menjadi Reading Time
# df_resampled = (
#     df1.set_index("Reading Time")
#        .resample("30T", origin=start_time)
#        .mean(numeric_only=True)
#        .reset_index()
# )
# Pastikan kolom Reading Time sudah datetime
df1["Reading Time"] = pd.to_datetime(df1["Reading Time"])

# Resample ke setiap 30 menit, align ke 00 dan 30 menit
df_resampled = (
    df1.set_index("Reading Time")
       .resample("30T", origin="epoch")   # align ke 00, 30
       .mean(numeric_only=True)
       .reset_index()
)

In [9]:
df_resampled.head()

,Reading Time,Average Amps (A) (Raw),Drive Frequency (Hz) (Raw),Intake Pressure (psi) (Raw),Discharge Pressure (psi) (Raw),Intake Temperature (F) (Raw),Motor Temperature (F) (Raw),Vibration (gravit) (Raw),Virtual Rate (BFPD) (Raw)
0,2025-03-03 17:00:00,0.0,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
1,2025-03-03 17:30:00,0.0,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
2,2025-03-03 18:00:00,0.0,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
3,2025-03-03 18:30:00,0.0,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0
4,2025-03-03 19:00:00,0.0,0.0,0.0,0.0,-4.000000e-09,-4.000000e-09,0.0,0.0


In [10]:
slopes_df.head()

,A,IP,DP,IT,MT,V,R
Window_Start_Time,,,,,,,
2025-03-03 17:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-03-03 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-03-03 18:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-03-03 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-03-03 19:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
slopes_df = slopes_df.reset_index()
X_predict = slopes_df.drop(columns=['Window_Start_Time'])

# Display the head of the feature matrix
display(X_predict.head())

,A,IP,DP,IT,MT,V,R
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
slopes_df.to_csv('slopes_df_30menit.csv', index=False)
X_predict.to_csv('X_predict_30menit.csv', index=False)
df_resampled.to_csv('df_all.csv', index=False)